In [5]:
from metakernel import register_ipython_magics
register_ipython_magics()

import ipywidgets as widgets
import sys
from IPython.display import display
from IPython.display import clear_output

from pythonds.basic import Stack

def create_multipleChoice_widget(description, options, correct_answer):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    radio_options = [(words, i) for i, words in enumerate(options)]
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False
    )
    
    description_out = widgets.Output()
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "Richtig." + '\x1b[0m' +"\n" #green color
        else:
            s = '\x1b[5;30;41m' + "Fail. " + '\x1b[0m' +"\n" #red color
        with feedback_out:
            clear_output()
            print(s)
        return
    
    check = widgets.Button(description="submit")
    check.on_click(check_selection)
    
    
    return widgets.VBox([description_out, alternativ, check, feedback_out])


frage1 = 'Wie ist der Wert wenn die dezimale Zahl 25 als oktaler Wert dargestellt wird?'
frage2 ='Wie ist der Wert der hexadezimalen Zahl AB in eine Zahl zur Basis 13?'
frage3 ='Welchen Wert hat die dezimale Zahl <n> in einer Zahl zur Basis <n>? <n> ist beliebig, aber fest'
frage4 ='Welchen Wert muss x im folgende Postfix-Ausdruck besitzen, damit er True ist? 17 10 + 3 * 9 / == x'
frage5 ='Erweitern Sie die Funktion infixToPostfix, so dass der Infix-Ausdruck 5*3^(4-2) ausgewertet wird und geben Sie diesen an'
       
Q1 = create_multipleChoice_widget(frage1,['19','31','25','42'],'31')
Q2 = create_multipleChoice_widget(frage2,['102','FF','171','nicht berechenbar'],'102')
Q3 = create_multipleChoice_widget(frage3, ['nicht berechenbar','10','00','11'],'10')
Q4 = create_multipleChoice_widget(frage4, ['X','1','9','False'],'9')
Q5 = create_multipleChoice_widget(frage5, ['534-2^*','53*42-^','5342-^*','53^42-*'],'5342-^*')


ModuleNotFoundError: No module named 'metakernel'

[<img src="Bilder/MIREVIBanner.jpg">](../../FMA_start_here.ipynb)

# Abstrakte Datenstrukturen, Teil 2
Teil 2 der interaktiven FMA-Lerneinheit zu abstrakten Datenstrukturen

## Ziele 
- die abstrakten Datentypen Stack, Queue, Deque und List verstehen.
- die ADTs Stack, Queue und Deque unter Verwendung von Python-Listen implementieren können.
- **die Performanz der Implementierungen von grundlegenden linearen Datenstrukturen zu verstehen.**
- **Verstehen der Präfix-, Infix- und Postfix-Ausdrucksformate.**
- **Verwendung von Stacks zur Auswertung von Postfix-Ausdrücken.**
- **Verwendung von Stacks zur Konvertierung von Ausdrücken von Infix- in Postfix-Ausdrücke.**
- Verwendung von Warteschlangen für grundlegende Timing-Simulationen.
- Probleme erkennen, bei denen Stacks, Warteschlangen und Deques geeignete Datenstrukturen sind.
- den ADT Liste als verlinkte Liste unter Verwendung des Knoten- und Zeigerkonzepts implementieren zu können.
- die Leistung der Implementierung der verknüpften Liste mit der Listenimplementierung von Python vergleichen.
---


## Beispiel 1 für ADT: Balancierte Klammern (Teil 1: Einfache Struktur)

Wir wenden uns nun der Verwendung von Stacks zur Lösung realer Informatikprobleme zu. Sie haben zweifellos schon arithmetische Ausdrücke definiert wie

$(5+6)∗(7+8)/(4+3)$

$(a+b)^2-a(x^3-5(x^2-2))$

In solchen Beispielen müssen die Klammern in *ausgewogener Weise* erscheinen. Ausgewogene Klammern bedeuten, dass jedes Anfangssymbol ein entsprechendes Schlusssymbol hat und die Klammernpaare richtig verschachtelt sind. Beachten Sie die folgenden korrekt ausgewogenen Klammerfolgen:
````
(()()()())
(((())))
(()((())()))
````
Vergleichen Sie diese mit den folgenden, die nicht ausgewogen sind:
```
((((((())
()))
(()()(()
```   

Die Fähigkeit, zwischen korrekt ausbalancierten und unausbalancierten Klammern zu unterscheiden, ist ein wichtiger Teil der Erkennung vieler Programmiersprachenstrukturen. Die Herausforderung besteht dann darin, einen Algorithmus zu schreiben, der eine Reihe von Klammern von links nach rechts liest und entscheidet, ob die Symbole ausgewogen sind. Um dieses Problem zu lösen, müssen wir eine wichtige Beobachtung machen. Wenn Sie Symbole von links nach rechts verarbeiten, muss die innerste öffnende Klammer mit dem nächsten schließenden Symbol übereinstimmen (siehe Abbildung 1). Außerdem muss das erste zu verarbeitende Eröffnungssymbol möglicherweise bis zum allerletzten Symbol warten, bis es aufgelöst werden kann. Die Schlusssymbole stimmen mit den Eröffnungssymbolen in der umgekehrten Reihenfolge ihres Auftretens überein; sie stimmen von innen nach außen überein. Dies ist auch ein Hinweis darauf, dass der ADT Stapel zur Lösung des Klammer-Problems verwendet werden kann.

<img src="Bilder/ADT/simpleparcheck.png" width=400px>
<center><b>Abbildung 1:</b> Test von einfachen Klammerstrukturen</center>

Wenn zur Problemlösung die ADT-Struktur *Stack* als geeignete Datenstruktur für die Beibehaltung der Klammern eingesetzt werden soll, ist die Funktionsweise des Algorithmus relativ einleuchtend. Beginnen Sie mit einem leeren Stack und verarbeiten Sie die Klammerzeichenfolgen von links nach rechts. Wenn es sich bei einem Symbol um eine öffnende Klammer handelt, schieben Sie es auf den Stapel als Signal, dass später ein entsprechendes schließendes Symbol erscheinen muss. Wenn es sich bei einem Symbol dagegen um eine schließende Klammer handelt wird das letzte Element des Stacks durch ``pop``gelöscht. Solange dies ohne Fehlermeldung immer bei jedem schließenden Symbol aufzubrechen, ist die Klammerstruktur ausgeglichen. Wenn sich zu irgendeinem Zeitpunkt kein Eröffnungssymbol auf dem Stapel befindet, das mit einem Schlusssymbol übereinstimmt, ist die Zeichenfolge nicht korrekt ausbalanciert. Am Ende der Zeichenfolge, wenn alle Symbole verarbeitet worden sind, muss der Stapel leer sein. Der Python-Code zur Implementierung dieses Algorithmus wird in *Listing 3* gezeigt.

**Codebeispiel 1. Das Problem der ausgeglichenen Klammern lösen**

In [ ]:
def parChecker(symbolString):
    s = Stack()
    balanced = True
    index = 0
    while index < len(symbolString) and balanced:
        symbol = symbolString[index]
        if symbol == "(":
            s.push(symbol)
        else:
            if s.isEmpty():
                balanced = False
            else:
                s.pop()

        index = index + 1

    if balanced and s.isEmpty():
        return True
    else:
        return False

print(parChecker('((()))'))
print(parChecker('(()'))

Diese Funktion, ``parChecker``, geht davon aus, dass eine ``Stack``-Klasse verfügbar ist und gibt ein boolesches Ergebnis zurück, ob die Klammerzeichenfolge ausgeglichen ist. Beachten Sie, dass die boolesche Variable ``balanced`` auf ``True`` initialisiert wird, da es keinen Grund gibt, am Anfang etwas anderes anzunehmen. Wenn das aktuelle Symbol ``(`` ist, dann wird es auf den Stapel geschoben (Zeilen 9-10). Beachten Sie auch in Zeile 15, dass ``pop`` einfach ein Symbol vom Stapel entfernt. Der zurückgegebene Wert wird nicht verwendet, da wir wissen, dass es sich um ein Eröffnungssymbol handeln muss, das wir schon einmal gesehen haben. Am Ende (Zeilen 19-22) stellt die Zeichenfolge eine korrekt ausbalancierte Folge von Klammern dar, solange der Ausdruck ausgeglichen ist und der Stapel vollständig geleert wurde.

---

## Balancierte Klammern (Teil 2: Allgemeiner Fall)

Das oben gezeigte Problem der ausgewogenen Klammerung ist ein spezieller Fall einer allgemeineren Situation, die in vielen Programmiersprachen auftritt. Das allgemeine Problem der Ausgewogenheit und Verschachtelung verschiedener Arten von Symbolen zum Öffnen und Schließen tritt häufig auf. Beispielsweise werden in Python eckige Klammern [ und ] für Listen, geschweifte Klammern { und } für Wörterbücher und Klammern ( und ) für Tupel und arithmetische Ausdrücke verwendet. Es ist möglich, Symbole zu mischen, solange jedes seine eigene offene und enge Beziehung zueinander unterhält. Zeichenfolgen von Symbolen wie

```
{ { ( [ ] [ ] ) } ( ) }

[ [ { { ( ( ) ) } } ] ]

[ ] [ ] [ ] ( ) { }
```

sind insofern ausgewogen, als nicht nur jedes Eröffnungssymbol ein entsprechendes Schlusssymbol hat, sondern auch die Arten der Symbole übereinstimmen. Vergleichen Sie diejenigen mit den folgenden Zeichenfolgen, die nicht ausgeglichen sind:

```
( [ ) ]

( ( ( ) ] ) )

[ { ( ) ]
```

Der einfache Klammer-Checker aus dem vorigen Abschnitt kann leicht erweitert werden, um diese neuen Arten von Symbolen zu handhaben. Erinnern Sie sich, dass jedes Eröffnungssymbol einfach auf den Stapel geschoben wird, um darauf zu warten, dass das passende Schlusssymbol später in der Sequenz erscheint. Wenn ein Schlusssymbol erscheint, besteht der einzige Unterschied darin, dass wir überprüfen müssen, ob es korrekt mit dem Typ des Eröffnungssymbols oben auf dem Stapel übereinstimmt. Wenn die beiden Symbole nicht übereinstimmen, ist die Zeichenfolge nicht ausgeglichen. Auch hier gilt: Wenn die gesamte Zeichenfolge verarbeitet wird und nichts mehr auf dem Stapel übrig ist, ist die Zeichenfolge korrekt ausbalanciert.

## Aufgabe: Modifikation des Programm zur Nutzung allgemeiner Klammerstrukturen

Das Python-Programm, das dies implementieren soll, wird in **Listing 4** gezeigt und soll von Ihnen vervollständigt werden. Die wesentliche Änderung zur bisherigen Version in **Listing 3** erscheint ab Zeile 16, wo wir eine Hilfsfunktion, ``matches``, schreiben müssen, die aufgerufen wird, um zu prüfen ob die übergebene schließende Klammer der richtigen öffnenden Klammer auf dem Stack entspricht 

die Symbolanpassung zu unterstützen. Jedes Symbol, das vom Stapel entfernt wird, muss überprüft werden, um sicherzustellen, dass es mit dem aktuellen schließenden Symbol übereinstimmt. Wenn eine Nichtübereinstimmung auftritt, wird die boolesche Variable ``balanced`` auf ``False`` gesetzt.

**Listing 4** Lösung des allgemeinen Klammer-Beispiels (parcheck2)
```python
def parChecker(symbolString):
    s = Stack()
    balanced = True
    index = 0
    while index < len(symbolString) and balanced:
        symbol = symbolString[index]
        
        # TO DO: check if symbol is any opening parantheses
        #if symbol in ....
        #    s.push(symbol)
        #else:
        #    if s.isEmpty():
        #        balanced = False
        #    else:
        #
        # TO DO: check if the symbol on top matches the currently processed symbol
        #        ...    # get the symbol from the top of the stack
        #        ...    # if it does not match, set balanced to False
        #               balanced = False

        index = index + 1
    if balanced and s.isEmpty():
        return True
    else:
        return False

# TO DO: Define a function 'matches' that checks if the symbol open correspond to symbol close
# i.e. "(" matches ")", "[" matches "]" and "{" matches "}"
# def matches(open,close):
#    
#    return ???


print(parChecker('{({([][])}())}'))  # should return True
print(parChecker('[{()]'))  # should return False
```

In [ ]:
# solution
def parChecker(symbolString):
    s = Stack()
    balanced = True
    index = 0
    while index < len(symbolString) and balanced:
        symbol = symbolString[index]
        if symbol in "([{":
            s.push(symbol)
        else:
            if s.isEmpty():
                balanced = False
            else:
                top = s.pop()
                if not matches(top,symbol):
                       balanced = False
        index = index + 1
    if balanced and s.isEmpty():
        return True
    else:
        return False

def matches(open,close):
    opens = "([{"
    closers = ")]}"
    return opens.index(open) == closers.index(close)

#print(parChecker('{({([][])}())}'))
#print(parChecker('[{()]'))

In [ ]:
# Start Coding here

---

## Beispiel 2 zu ADT: Konvertierung von Dezimalzahlen zu Binärzahlen
---


In Ihrem Studium der Informatik sind Sie wahrscheinlich auf die eine oder andere Weise mit der Idee einer binären Zahl in Berührung gekommen. Die Binärdarstellung ist in der Informatik wichtig, da alle Werte, die in einem Computer gespeichert sind, als eine Folge von Binärziffern, eine Folge von 0s und 1s, existieren. Ohne die Fähigkeit, zwischen gängigen Darstellungen und Binärzahlen hin und her zu konvertieren, müssten wir mit Computern auf sehr umständliche Weise interagieren.

Ganzzahlige Werte sind übliche Datenelemente. Sie werden in Computerprogrammen und beim Rechnen ständig verwendet. Wir lernen sie im Matheunterricht kennen und stellen sie natürlich mit dem Dezimalzahlensystem oder zur Basis 10 dar. Die Dezimalzahl 23310 und ihr entsprechendes binäres Äquivalent 111010012 werden jeweils interpretiert als

$2×10^2+3×10^1+3×10^0$

und

$1×2^7+1×2^6+1×2^5+0×2^4+1×2^3+0×2^2+0×2^1+1×2^0$

Aber wie können wir ganzzahlige Werte leicht in Binärzahlen umwandeln? Die Antwort ist ein Algorithmus namens "Dividieren durch 2", der einen Stapel verwendet, um die Ziffern für das Binärergebnis zu verfolgen.

Der Algorithmus "Dividieren durch 2" geht davon aus, dass wir mit einer ganzen Zahl größer als 0 beginnen. Eine einfache Iteration teilt dann kontinuierlich die Dezimalzahl durch 2 und behält den Rest im Auge. Die erste Division durch 2 gibt Auskunft darüber, ob der Wert gerade oder ungerade ist. Ein gerader Wert hat einen Rest von 0 und die Ziffer 0 an der Einerstelle. Ein ungerader Wert hat einen Rest von 1 und hat die Ziffer 1 an der Einerstelle. Wir denken darüber nach, unsere Binärzahl als eine Folge von Ziffern aufzubauen; der erste Rest, den wir berechnen, wird tatsächlich die letzte Ziffer in der Folge sein. Wie in *Abbildung 5* dargestellt, sehen wir wieder die Umkehreigenschaft, die signalisiert, dass ein Stack wahrscheinlich die geeignete Datenstruktur für die Lösung des Problems ist.

<img src="Bilder/ADT/dectobin.png" width=600px>
<center><b>Abbildung 2:</b> Konvertierung Dezimalzahl in Binärzahl</center>

**Codebeispiel 2. Funktion divideBy2 zur Konvertierung in eine Binärzahl**

In [ ]:
def divideBy2(decNumber):
    remstack = Stack()

    while decNumber > 0:
        rem = decNumber % 2
        remstack.push(rem)
        decNumber = decNumber // 2

    binString = ""
    while not remstack.isEmpty():
        binString = binString + str(remstack.pop())

    return binString

die_zahl = 42
print(die_zahl,"ist in Binärschreibweise",divideBy2(42))

Der Algorithmus für die Binärkonvertierung kann leicht erweitert werden, um die Konvertierung für jede beliebige Basis durchzuführen. In der Informatik ist es üblich, eine Reihe verschiedener Kodierungen zu verwenden. Die gebräuchlichsten davon sind binär, oktal (Basis 8) und hexadezimal (Basis 16).

Die Dezimalzahl 233 und ihre entsprechenden oktalen und hexadezimalen Äquivalente 3518 und 𝐸916 werden interpretiert als

$3×8^2+5×8^1+1×8^0$

und

$14×16^1+9×16^0$

Die Funktion ``dividideBy2`` kann so modifiziert werden, dass sie nicht nur einen dezimalen Wert, sondern auch eine Basis für die beabsichtigte Konvertierung akzeptiert. Die Idee "Dividieren durch 2" wird einfach durch eine allgemeinere Idee "Dividieren durch Basis" ersetzt. Eine neue Funktion namens ``baseConverter``, die in **Listing 6** entwickelt werden soll, nimmt eine Dezimalzahl und eine beliebige Basis zwischen 2 und 16 als Parameter an. Die Reste werden immer noch auf den Stapel geschoben, bis der umzuwandelnde Wert 0 wird. Mit einer geringfügigen Änderung kann die gleiche Technik zur Konstruktion von Strings von links nach rechts verwendet werden. Zahlen zur Basis 2 bis zur Basis 10 benötigen maximal 10 Stellen, so dass die typischen Stellenzeichen 0, 1, 2, 3, 4, 5, 6, 7, 8 und 9 gut funktionieren. Das Problem entsteht, wenn wir über die Basis 10 hinausgehen. Wir können die Reste nicht mehr einfach verwenden, da sie selbst als zweistellige Dezimalzahlen dargestellt werden. Stattdessen müssen wir einen Satz von Ziffern schaffen, mit denen wir die Reste jenseits von 9 darstellen können.


## Aufgabe: Modifikation des Listing 5 um eine Dezimalzahl in eine beliebige Basis zu transformieren

Erweitern Sie das Programm aus **Codebeipiel 2** um eine Transformation in eine beliebige Basis zu ermöglichen. Dazu muss der Algorithmus wie folgt modifiziert werden:
- Division findet durch die Basis base statt (statt durch 2)
- der Rest beim Teilen muss als geeignete Ziffer des Zahlensystems dargestellt werden, also zB A,B,C,D,E,F für 10,11,12,13,14,15 bei Basis 16

**Listing 6. Basisstruktur Konvertierung einer Dezimalzahl in allgemeine Basiszahlen (bis 16)**

```python
from pythonds.basic import Stack

def baseConverter(decNumber,base):
    digits = "0123456789ABCDEF"

    remstack = Stack()

    while decNumber > 0:
        # write new function here
      

    newString = ""
    while not remstack.isEmpty():
        # write new function here
        
    return newString

print(baseConverter(25,2))  # sollte 11001 liefern
print(baseConverter(25,13)) # sollte 1C liefern
```

**Codebeispiel 4. Eigene Lösung**

In [ ]:
from pythonds.basic import Stack

def baseConverter(decNumber,base):
    #
    #
    newString = ''
    #   
    return newString

print(baseConverter(25,2))  # sollte 11001 liefern
print(baseConverter(25,13)) # sollte 1C liefern

Eine Lösung für dieses Problem besteht darin, den Ziffernsatz um einige Buchstaben des Alphabets zu erweitern. Hexadezimal verwendet zum Beispiel die zehn Dezimalziffern zusammen mit den ersten sechs Buchstaben des Alphabets für die 16 Ziffern. Um dies zu implementieren, wird eine Ziffernfolge erstellt (Zeile 4 in Listing 6), die die Ziffern an ihren entsprechenden Positionen speichert. 0 steht an Position 0, 1 steht an Position 1, A steht an Position 10, B steht an Position 11 und so weiter. Wenn ein Rest vom Stapel entfernt wird, kann er zum Indexieren in die Ziffernfolge verwendet werden, und die korrekte resultierende Ziffer kann an die Antwort angehängt werden. Wenn zum Beispiel der Rest 13 vom Stapel entfernt wird, wird die Ziffer D an die resultierende Zeichenfolge angehängt.

---

### ADT-03 Selbstest

#### Programmieraufgabe ####

Ändern Sie die Listings in diesem Abschnitt so ab, dass eine Transformation von einer beliebigen Basis A (2..16) zu einer anderen beliebigen Basis B (2..16) erfolgt.

**Listing 7. Konvertierung einer Zahl von und zu einer beliebigen Basis (2..16)**
```python
from pythonds.basic import Stack

def baseConverter(Number,fromBase,toBase):
    digits = "0123456789ABCDEF"

    remstack = Stack()

    while Number > 0:
        # write new function here
      

    newString = ""
    while not remstack.isEmpty():
        # write new function here
        
    return newString

print(baseConverter(25,10,2))  # sollte 11001 liefern
print(baseConverter(FE,16,13)) # sollte 167 liefern
```

**Codebeispiel 5. Eigene Lösung**

In [ ]:
from pythonds.basic import Stack

def baseConverter(Number,fromBase,toBase):
    #
    #
    #  Lösung hier implementieren
    #
    #
    newString = ""
    #   
    return newString

print(baseConverter('25','10','2'))  # sollte 11001 liefern
print(baseConverter('FE','16','13')) # sollte 167 liefern

In [ ]:
display(Q1)
display(Q2)
display(Q3)

---
## Beispiel 3: Infix, Prefix and Postfix Ausdrücke
---

Wenn Sie einen arithmetischen Ausdruck wie $B*C$ schreiben, liefert Ihnen die Form des Ausdrucks Informationen, damit Sie ihn richtig interpretieren können. In diesem Fall wissen wir, dass die Variable $B$ mit der Variablen $C$ multipliziert wird, da der Multiplikationsoperator $*$ im Ausdruck dazwischen steht. Diese Art der Notation wird als *Infix* bezeichnet, da der Operator *zwischen* den beiden Operanden steht, an denen er arbeitet.

Betrachten Sie ein anderes Beispiel für ein Infix, $A + B * C$. Die Operatoren $+$ und $*$ erscheinen immer noch zwischen den Operanden, aber es gibt ein Problem. An welchen Operanden arbeiten sie? Arbeitet das $+$ auf $A$ und $B$ oder nimmt das $*$ $B$ und $C$? Der Ausdruck scheint mehrdeutig zu sein.

Tatsächlich lesen und schreiben Sie diese Art von Ausdrücken schon seit langer Zeit und sie bereiten Ihnen keine Probleme. Der Grund dafür ist, dass Sie etwas über die Operatoren + und * wissen. Jeder Operator hat eine Prioritätsstufe. Operatoren mit höherer Priorität werden vor Operatoren mit niedrigerer Priorität verwendet. Das Einzige, was diese Reihenfolge ändern kann, ist das Vorhandensein von Klammern. Die Rangfolge der arithmetischen Operatoren stellt Multiplikation und Division über Addition und Subtraktion. Wenn zwei gleichrangige Operatoren auftreten, wird eine von links nach rechts gerichtete Ordnung oder Assoziativität verwendet.

Lassen Sie uns den lästigen Ausdruck $A + B * C$ mit Hilfe der Operatorpräferenz interpretieren. $B$ und $C$ werden zuerst multipliziert, und dann wird $A$ zu diesem Ergebnis addiert. $(A + B) * C$ würde erzwingen, dass die Addition von $A$ und $B$ zuerst durchgeführt wird, bevor die Multiplikation erfolgt. Im Ausdruck $A + B + C$ würde bei der Präzedenz (über die Assoziativität) das ganz linke $+$ zuerst ausgeführt werden.

Auch wenn all dies für Sie offensichtlich sein mag, denken Sie daran, dass Computer genau wissen müssen, welche Operatoren in welcher Reihenfolge ausgeführt werden müssen. Eine Möglichkeit, einen Ausdruck zu schreiben, der garantiert, dass es keine Verwirrung bezüglich der Reihenfolge der Operationen gibt, besteht darin, einen so genannten vollständig eingeklammerten Ausdruck zu erstellen. Diese Art von Ausdruck verwendet ein Paar Klammern für jeden Operator. Die Klammern diktieren die Reihenfolge der Operationen; es gibt keine Mehrdeutigkeit. Es gibt auch keine Notwendigkeit, sich an irgendwelche Prioritätsregeln zu erinnern.

Der Ausdruck $A + B * C + D$ kann wie folgt umgeschrieben werden $((A + (B * C)) + D)$, um zu zeigen, dass die Multiplikation zuerst erfolgt, gefolgt von der Addition ganz links. $A + B + C + D$ kann als $(((A + B) + C) + D)$ geschrieben werden, da die Additionsoperationen von links nach rechts assoziieren.

Es gibt zwei weitere sehr wichtige Ausdrucksformate, die Ihnen vielleicht zunächst nicht offensichtlich erscheinen. Denken Sie an den Infix-Ausdruck $A + B$. Was würde passieren, wenn wir den Operator vor die beiden Operanden stellen? Der resultierende Ausdruck wäre $+ A B$. Ebenso könnten wir den Operator an das Ende verschieben. Wir würden $A B +$ erhalten. Diese sehen etwas seltsam aus.

Diese Änderungen der Position des Operators in Bezug auf die Operanden erzeugen zwei neue Ausdrucksformate, Präfix und Postfix. Die Notation von Präfixausdrücken erfordert, dass alle Operatoren vor den beiden Operanden stehen, an denen sie arbeiten. Die Postfix-Notation hingegen verlangt, dass ihre Operatoren nach den entsprechenden Operanden stehen. Einige weitere Beispiele sollen dazu beitragen, dies etwas klarer zu machen (siehe Tabelle 2).

$A + B * C$ würde als $+ A * B C$ im Präfix geschrieben werden. Der Multiplikationsoperator steht unmittelbar vor den Operanden $B$ und $C$, was bedeutet, dass $*$ Vorrang vor $+$ hat. Der Additionsoperator steht dann vor dem $A$ und dem Ergebnis der Multiplikation.

Im Postfix würde der Ausdruck $A B C * +$ sein. Auch hier bleibt die Reihenfolge der Operationen erhalten, da das  $*$ unmittelbar nach dem $B$ und dem $C$ steht, was bedeutet, dass $*$ Vorrang hat, wobei $+$ hinter $+$ steht. Obwohl die Operatoren verschoben wurden und nun entweder vor oder nach ihren jeweiligen Operanden erscheinen, blieb die Reihenfolge der Operanden relativ zueinander genau gleich. 

<img src="Bilder/ADT/table_postfix.png" width=600px>
<center><b>Tabelle 1:</b> Beispiele für Präfix, Infix und Postfix-Ausdrücke</center>


Betrachten Sie nun den Infix-Ausdruck $(A + B) * C$. Denken Sie daran, dass Infix in diesem Fall die Klammern benötigt, um die Durchführung der Addition vor der Multiplikation zu erzwingen. Wenn jedoch $A + B$ in Präfix geschrieben wurde, wurde der Additionsoperator einfach vor die Operanden $+ A B$ geschoben. Das Ergebnis dieser Operation wird der erste Operand für die Multiplikation. Der Multiplikationsoperator wird vor den gesamten Ausdruck geschoben, so dass wir $* + A B C$ erhalten. Ebenso erzwingt im Postfix $A B +$, dass die Addition zuerst erfolgt. Die Multiplikation kann mit diesem Ergebnis und dem verbleibenden Operanden C durchgeführt werden. Der richtige Postfix-Ausdruck ist dann $A B + C *$.

Betrachten Sie diese drei Ausdrücke noch einmal (siehe Tabelle 3). Etwas sehr Wichtiges ist geschehen. Wo sind die Klammern hin? Warum brauchen wir sie nicht in Präfix und Postfix? Die Antwort ist, dass die Operatoren in Bezug auf die Operanden, an denen sie arbeiten, nicht mehr zweideutig sind. Nur die Infix-Notation erfordert die zusätzlichen Symbole. Die Reihenfolge der Operationen innerhalb von Präfix- und Postfix-Ausdrücken wird vollständig durch die Position des Operators und durch nichts anderes bestimmt. In vielerlei Hinsicht macht dies die Infix-Notation zur am wenigsten wünschenswerten Notation.

<img src="Bilder/ADT/Klammerausdruck.png" width=600px>
<center><b>Tabelle 3:</b> Gleicher Klammerausdruck in verschiedenen Notationen</center>

Tabelle 4 zeigt einige zusätzliche Beispiele von Infix-Ausdrücken und den entsprechenden Präfix- und Postfix-Ausdrücken. Vergewissern Sie sich, dass Sie verstehen, inwiefern sie in Bezug auf die Reihenfolge der ausgeführten Operationen äquivalent sind.

<img src="Bilder/ADT/TabelleAusdruecke.png" width=600px>
<center><b>Tabelle 4:</b> Weitere Beispiele verschiedenen Notationen</center>

---

## Infix-Konvertierung zu Prefix und Postfix

Bisher haben wir Ad-hoc-Methoden zur Konvertierung zwischen Infix-Ausdrücken und den entsprechenden Präfix- und Postfix-Ausdrucksnotationen verwendet. Wie zu erwarten ist, gibt es algorithmische Verfahren zur Durchführung der Konvertierung, die es ermöglichen, jeden Ausdruck beliebiger Komplexität korrekt zu transformieren.

Die erste Technik, die wir betrachten, verwendet das Konzept eines vollständig eingeklammerten Ausdrucks, das bereits früher diskutiert wurde. Es sei daran erinnert, dass $A + B * C$ als $(A + (B * C))$ geschrieben werden kann, um ausdrücklich zu zeigen, dass die Multiplikation Vorrang vor der Addition hat. Bei näherer Betrachtung können Sie jedoch erkennen, dass jedes Klammerpaar auch den Anfang und das Ende eines Operandenpaares mit dem entsprechenden Operator in der Mitte bezeichnet.

Sehen Sie sich die rechte Klammer im Unterausdruck $(B * C)$ oben an. Wenn wir das Multiplikationssymbol an diese Stelle verschieben und die entsprechende linke Klammer entfernen würden, so dass wir $B C *$ erhielten, hätten wir den Unterausdruck in der Tat in die Postfix-Notation umgewandelt. Würde der Additionsoperator ebenfalls an die entsprechende Position der rechten Klammer verschoben und die passende linke Klammer entfernt werden, würde sich der vollständige Postfix-Ausdruck ergeben (siehe Abbildung 6).

<img src="Bilder/ADT/moveright.png" width=300px>
<center><b>Abbildung 5:</b> Aus Infix wird Postfix</center>

Wenn wir dasselbe tun, aber das Symbol nicht an die Position der rechten Klammer, sondern nach links verschieben, erhalten wir die Präfixnotation (siehe Abbildung 7). Die Position des Klammerpaares ist eigentlich ein Hinweis auf die Endposition des eingeschlossenen Operators.

<img src="Bilder/ADT/moveleft.png" width=300px>
<center><b>Abbildung 5:</b> Aus Infix wird Präfix</center>

Um also einen Ausdruck, egal wie komplex, entweder in Präfix- oder Postfixnotation umzuwandeln, klammern Sie den Ausdruck in der Reihenfolge der Operationen vollständig in Klammern. Dann schieben Sie den eingeschlossenen Operator an die Position der linken oder rechten Klammer, je nachdem, ob Sie Präfix- oder Postfixnotation wünschen.

Hier ist ein komplexerer Ausdruck: $(A + B) * C - (D - E) * (F + G)$. Abbildung 6 zeigt die Umwandlung in Postfix- und Präfixnotation.


<img src="Bilder/ADT/complexmove.png" width=500px>

<center><b>Abbildung 6:</b> Komplexes Beispiel für Umwandlung in Postfix und Präfix</center>

---

## Allgemeine Konvertierung Infix2Posfix

Wir müssen einen Algorithmus entwickeln, um einen beliebigen Infix-Ausdruck in einen Postfix-Ausdruck umzuwandeln. Dazu werden wir uns den Konvertierungsprozess genauer ansehen.

Betrachten wir noch einmal den Ausdruck A + B * C. Wie oben gezeigt, ist A B C * + das Postfix-Äquivalent. Wir haben bereits festgestellt, dass die Operanden A, B und C in ihren relativen Positionen bleiben. Nur die Operatoren ändern ihre Position. Schauen wir uns noch einmal die Operatoren im Infix-Ausdruck an. Der erste Operator, der von links nach rechts erscheint, ist +. Im Postfix-Ausdruck steht + jedoch am Ende, da der nächste Operator, *, Vorrang vor der Addition hat. Die Reihenfolge der Operatoren im ursprünglichen Ausdruck wird im resultierenden Postfix-Ausdruck umgekehrt.

Während wir den Ausdruck verarbeiten, müssen die Operatoren irgendwo gespeichert werden, da ihre entsprechenden rechten Operanden noch nicht gesehen werden. Auch die Reihenfolge dieser gespeicherten Operatoren muss aufgrund ihres Vorrangs unter Umständen umgekehrt werden. Dies ist in diesem Beispiel bei der Addition und der Multiplikation der Fall. Da der Additionsoperator vor dem Multiplikationsoperator steht und einen niedrigeren Vorrang hat, muss er nach dem Multiplikationsoperator erscheinen. Wegen dieser Umkehrung der Reihenfolge ist es sinnvoll, die Verwendung eines Stapels in Betracht zu ziehen, um die Operatoren so lange zu behalten, bis sie benötigt werden.

Was ist mit (A + B) * C? Erinnern Sie sich daran, dass A B + C * das Postfix-Äquivalent ist. Auch hier sehen wir bei der Verarbeitung dieses Infix-Ausdrucks von links nach rechts zuerst +. In diesem Fall, wenn wir * sehen, wurde + bereits in den Ergebnisausdruck gesetzt, weil es aufgrund der Klammern Vorrang vor * hat. Wir können nun damit beginnen zu sehen, wie der Konvertierungsalgorithmus arbeiten wird. Wenn wir eine linke Klammer sehen, speichern wir sie ab, um anzuzeigen, dass ein weiterer Operator mit hoher Priorität kommen wird. Dieser Operator muss warten, bis die entsprechende rechte Klammer erscheint, um seine Position zu bezeichnen (erinnern Sie sich an die vollständig eingeklammerte Technik). Wenn diese rechte Klammer erscheint, kann der Operator vom Stapel entfernt werden.

Wenn wir den Infix-Ausdruck von links nach rechts scannen, werden wir einen Stapel verwenden, um die Operatoren zu behalten. Dies liefert die Umkehrung, die wir im ersten Beispiel bemerkt haben. Oben auf dem Stapel wird immer der zuletzt gespeicherte Operator stehen. Jedes Mal, wenn wir einen neuen Operator lesen, müssen wir uns überlegen, wie dieser Operator im Vergleich zu den Operatoren, die sich bereits auf dem Stapel befinden, Vorrang hat.

Nehmen wir an, der Infix-Ausdruck ist eine durch Leerzeichen getrennte Zeichenkette. Die Token des Operators sind *, /, + und -, zusammen mit der linken und rechten Klammer, ( und ). Die Operanden-Token sind die Einzelzeichen-Bezeichner A, B, C usw. Die folgenden Schritte erzeugen eine Zeichenfolge von Token in Postfix-Reihenfolge.

- Erstellen Sie einen leeren Stapel namens opstack, um die Operatoren zu behalten. Erstellen Sie eine leere Liste für die Ausgabe.

- Konvertieren Sie den Eingabe-Infix-String in eine Liste, indem Sie die String-Methode split verwenden.

- Scannen Sie die Token-Liste von links nach rechts.

   - Wenn das Token ein Operand ist, hängen Sie ihn an das Ende der Ausgabeliste an.

   - Wenn es sich bei dem Token um eine linke Klammer handelt, schieben Sie sie auf den Opstack.

   - Wenn es sich bei dem Token um eine rechte Klammer handelt, öffnen Sie den Opstack, bis die entsprechende linke Klammer entfernt ist. Hängen Sie jeden Operator an das Ende der Ausgabeliste an.

   - Wenn es sich bei dem Token um einen Operator *, /, + oder - handelt, schieben Sie ihn auf den Opstack. Entfernen Sie jedoch zuerst alle bereits auf dem Opstack befindlichen Operatoren mit höherer oder gleicher Priorität und hängen Sie sie an die Ausgabeliste an.

- Wenn der Eingabeausdruck vollständig verarbeitet wurde, überprüfen Sie den Opstack. Alle noch auf dem Stapel befindlichen Operatoren können entfernt und an das Ende der Ausgabeliste angehängt werden.

Abbildung 7 zeigt den Konvertierungsalgorithmus, der mit dem Ausdruck A * B + C * D arbeitet. Beachten Sie, dass der erste Operator * entfernt wird, sobald der Operator + zu sehen ist. Außerdem bleibt + auf dem Stapel, wenn der zweite * auftritt, da die Multiplikation Vorrang vor der Addition hat. Am Ende des Infix-Ausdrucks wird der Stapel zweimal gepoppt, wobei beide Operatoren entfernt und + als letzter Operator im Postfix-Ausdruck platziert wird. 


<img src="Bilder/ADT/intopost.png" width=600px>
<center><b>Abbildung 7:</b> Algorithmus für Infix zu Postfix-Transformation</center>

Um den Algorithmus in Python zu kodieren, werden wir ein Wörterbuch namens ``prec`` verwenden, um die Prioritätswerte für die Operatoren festzuhalten. Dieses Wörterbuch wird jeden Operator auf eine ganze Zahl abbilden, die mit den Prioritätsstufen anderer Operatoren verglichen werden kann (wir haben willkürlich die ganzen Zahlen 3, 2 und 1 verwendet). Die linke Klammer erhält den niedrigsten möglichen Wert. Auf diese Weise hat jeder Operator, der mit ihm verglichen wird, eine höhere Priorität und wird über ihm platziert. Zeile 15 definiert die Operanden als beliebige Großbuchstaben oder Ziffern. Die vollständige Konvertierungsfunktion ist in Codebeispiel 6 dargestellt.

**Codebeispiel 6. Konvertierungsfunktion für Postfix-Ausdrücke**

In [ ]:
from pythonds.basic import Stack

def infixToPostfix(infixexpr):
    prec = {}
    prec["*"] = 3
    prec["/"] = 3
    prec["+"] = 2
    prec["-"] = 2
    prec["("] = 1
    opStack = Stack()
    postfixList = []
    tokenList = infixexpr.split()

    for token in tokenList:
        if token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" or token in "0123456789":
            postfixList.append(token)
        elif token == '(':
            opStack.push(token)
        elif token == ')':
            topToken = opStack.pop()
            while topToken != '(':
                postfixList.append(topToken)
                topToken = opStack.pop()
        else:
            while (not opStack.isEmpty()) and \
               (prec[opStack.peek()] >= prec[token]):
                  postfixList.append(opStack.pop())
            opStack.push(token)

    while not opStack.isEmpty():
        postfixList.append(opStack.pop())
    return " ".join(postfixList)

print(infixToPostfix("A * B + C * D"))
print(infixToPostfix("( A + B ) * C - ( D - E ) * ( F + G )"))
infixToPostfix("( A + B ) * C")

In [ ]:
print(infixToPostfix("( A + B ) * C"))
#'A B + C *'
print(infixToPostfix("A + B * C"))
#'A B C * +'

## Postfix Auswertung

Als letztes Stack-Beispiel betrachten wir die Auswertung eines Ausdrucks, der sich bereits in der Postfix-Notation befindet. In diesem Fall ist ebenfalls ein Stack die Datenstruktur der Wahl. Wenn Sie jedoch den Postfix-Ausdruck scannen, müssen die Operanden warten, nicht die Operatoren wie im obigen Konvertierungsalgorithmus. Eine andere Möglichkeit, über die Lösung nachzudenken, besteht darin, dass immer dann, wenn ein Operator auf der Eingabe zu sehen ist, die beiden jüngsten Operanden in der Auswertung verwendet werden.

Um dies genauer zu sehen, betrachten Sie den Postfix-Ausdruck 4 5 6 * +. Wenn Sie den Ausdruck von links nach rechts scannen, treffen Sie zunächst auf die Operanden 4 und 5. Zu diesem Zeitpunkt sind Sie noch unsicher, was Sie mit ihnen machen sollen, bis Sie das nächste Symbol sehen. Durch das Ablegen auf dem Stapel wird sichergestellt, dass sie verfügbar sind, wenn ein Operator als nächstes kommt.

In diesem Fall ist das nächste Symbol ein weiterer Operand. Schieben Sie sie also wie zuvor und prüfen Sie das nächste Symbol. Jetzt sehen wir einen Operator, $*$. Dies bedeutet, dass die beiden jüngsten Operanden in einer Multiplikationsoperation verwendet werden müssen. Indem wir den Stapel zweimal springen lassen, können wir die richtigen Operanden erhalten und dann die Multiplikation durchführen (in diesem Fall erhalten wir das Ergebnis 30).

Wir können dieses Ergebnis nun behandeln, indem wir es wieder auf den Stapel legen, so dass es als Operand für die späteren Operatoren im Ausdruck verwendet werden kann. Wenn der letzte Operator verarbeitet wird, ist nur noch ein Wert auf dem Stapel übrig. Pop und gibt ihn als Ergebnis des Ausdrucks zurück. Abbildung 10 zeigt den Inhalt des Stapels, während dieser gesamte Beispielausdruck verarbeitet wird.

<img src="Bilder/ADT/evalpostfix1.png" width=600px>
<center><b>Abbildung 8:</b> ADT Stack mit Inhalten während der Ausführung</center>

Abbildung 11 zeigt ein etwas komplexeres Beispiel, 7 8 + 3 2 + / /. In diesem Beispiel sind zwei Dinge zu beachten. Erstens wächst und schrumpft die Stapelgröße und wächst dann wieder, wenn die Unterausdrücke ausgewertet werden. Zweitens, die Teilungsoperation muss sorgfältig gehandhabt werden. Erinnern Sie sich daran, dass die Operanden im Postfix-Ausdruck in ihrer ursprünglichen Reihenfolge stehen, da Postfix nur die Platzierung der Operatoren ändert. Wenn die Operanden für die Division vom Stapel entfernt werden, werden sie umgekehrt. Da es sich bei der Division nicht um einen kommutativen Operator handelt, d.h. 15/5 ist nicht dasselbe wie 5/15, müssen wir sicher sein, dass die Reihenfolge der Operanden nicht vertauscht wird.


<img src="Bilder/ADT/evalpostfix2.png" width=600px>
<center><b>Abbildung 8:</b> Komplexes Beispiel einer Evaluierung</center>

Angenommen, der Postfix-Ausdruck ist eine durch Leerzeichen abgegrenzte Zeichenfolge. Die Operatoren sind *, /, + und - und die Operanden werden als einstellige Ganzzahlwerte angenommen. Die Ausgabe wird ein ganzzahliges Ergebnis sein.

- Erstellen Sie einen leeren Stapel namens ``operandStack``.

- Wandeln Sie den String in eine Liste um, indem Sie die String-Methode ``split`` verwenden.

- Scannen Sie die Token-Liste von links nach rechts.

- Wenn das Token ein Operand ist, konvertieren Sie ihn von einer Zeichenkette in eine ganze Zahl und schieben Sie den Wert auf den ``operandStack``.

- Wenn das Token ein Operator, *, /, + oder - ist, benötigt es zwei Operanden. Legen Sie den `òperandStack`` zweimal ab. Der erste Pop ist der zweite Operand und der zweite Pop ist der erste Operand. Führen Sie die arithmetische Operation aus. Schieben Sie das Ergebnis auf den ``operandStack`` zurück.

- Wenn der Eingabeausdruck vollständig verarbeitet wurde, befindet sich das Ergebnis auf dem Stapel. Nimm mit ``pop``ein Element vom  ``operandStack`` auf und geben Sie den Wert zurück.

Die vollständige Funktion für die Auswertung von Postfix-Ausdrücken ist in *Listing 7* dargestellt. Zur Unterstützung der Arithmetik wird eine Hilfsfunktion ``doMath`` definiert, die zwei Operanden und einen Operator nimmt und dann die richtige arithmetische Operation ausführt.

**Codebeispiel 7 Postfix Auswertung eines Terms**

In [6]:
from pythonds.basic import Stack

def postfixEval(postfixExpr):
    operandStack = Stack()
    tokenList = postfixExpr.split()

    for token in tokenList:
        if token in "0123456789":
            operandStack.push(int(token))
        else:
            operand2 = operandStack.pop()
            operand1 = operandStack.pop()
            result = doMath(token,operand1,operand2)
            operandStack.push(result)
    return operandStack.pop()

def doMath(op, op1, op2):
    if op == "*":
        return op1 * op2
    elif op == "/":
        return op1 / op2
    elif op == "+":
        return op1 + op2
    else:
        return op1 - op2

print(postfixEval('7 8 + 3 2 + /'))

ModuleNotFoundError: No module named 'pythonds'

Es ist wichtig zu beachten, dass wir sowohl im Postfix-Konvertierungs- als auch im Postfix-Auswertungsprogramm davon ausgegangen sind, dass *keine Fehler* im Eingabeausdruck vorhanden sind. Wenn Sie diese Programme als Ausgangspunkt verwenden, können Sie leicht erkennen, wie Fehlererkennung und -meldung einbezogen werden können. Wir belassen dies als Übung am Ende des Kapitels.

### ADT-04 Selbstest

1. Konvertiere ohne die Nutzung des Programmcodes den folgenden Infix-Ausdruck nach Postfix 10 + 3 * 5 / (16 - 4) 

2. Ändern Sie die Funktion ``infixToPostfix`` so, dass sie den folgenden Ausdruck konvertieren kann: 5 * 3^(4 - 2). Führen Sie die Funktion auf dem Ausdruck aus und geben Sie die korrekte Lösung an:

In [ ]:
display(Q5)

Lösung als [Youtube-Video](https://youtu.be/LO-2q4pSsdM)

In [ ]:
# Eigene Lösug beginnt hier